In [1]:
%pip install datasets
import numpy
import transformers
import datasets


print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

Note: you may need to restart the kernel to use updated packages.
2.1.3
4.57.1
4.4.1


In [2]:
%pip install 'accelerate>=0.26.0'

Note: you may need to restart the kernel to use updated packages.


In [3]:

from datasets import load_dataset

In [4]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("e9t/nsmc", revision="refs/convert/parquet")

# split train into train and validation
split = dataset["train"].train_test_split(test_size=0.1, seed=42)

# wrap into a proper DatasetDict
hf_dataset = DatasetDict({
    "train": split["train"],
    "validation": split["test"],
    "test": dataset["test"]
})

In [5]:
cols = hf_dataset["train"].column_names   
cols





['id', 'document', 'label']

In [6]:
for i in range(5):
    for col in cols:
        print(f"{col}: {hf_dataset['train'][col][i]}")
    print("-----")

id: 5960467
document: 명작은 옛날과 현재의 동일한 공감대 위에 놓인다. 그리고 더스틴 호프만의 섹시미^^
label: 1
-----
id: 5244343
document: 원작에 미원을 치다.
label: 0
-----
id: 8786278
document: 경상도로 하고싶었지만 티날까봐강원도로 하셨나? 가만있는 강원도만불쌍하게 된겨?
label: 0
-----
id: 5452084
document: 나래이션 미쳤다 KBS 나레이션은 진짜 멋졌다!! 극장나래이션은 별점이아깝다!
label: 0
-----
id: 10045037
document: 영화 자체도 좋았지만. 아오이유우 때문에 모에사 할뻔했다.
label: 1
-----


In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)
 
# 모든 BERT 레이어 동결
for param in model.bert.parameters():
    param.requires_grad = False

# 상위 6개 레이어 unfreeze
num_layers_to_finetune = 6
total_layers = 12

for i in range(total_layers - num_layers_to_finetune, total_layers):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = True
    print(f"Layer {i} trainable")

# 분류기 항상 학습
for param in model.classifier.parameters():
    param.requires_grad = True

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer 6 trainable
Layer 7 trainable
Layer 8 trainable
Layer 9 trainable
Layer 10 trainable
Layer 11 trainable


In [8]:
def transform(data):
    return tokenizer(
        data["document"],
        truncation=True,
        padding="max_length",
        return_token_type_ids=False
    )

hf_dataset = hf_dataset.map(transform, batched=True)

hf_train_dataset = hf_dataset["train"]
hf_val_dataset = hf_dataset["validation"]
hf_test_dataset = hf_dataset["test"]


In [9]:
%pip install "transformers[torch]"
%pip install accelerate

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import numpy as np
from transformers import Trainer, TrainingArguments

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:


output_dir = 'transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    eval_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)



In [11]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [12]:
from evaluate import load
metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return metric.compute(predictions=preds, references=labels)

In [13]:
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=hf_dataset["train"],
    eval_dataset=hf_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)

trainer.train()

/var/folders/05/h8tbm7xs36jc1lx7bsvjgjcr0000gn/T/ipykernel_44125/2456982331.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/homebrew/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 